In [2]:
import pandas as pd

# 1. Finding Similar Costumers

# 2. Grouping customers together!

## 2.1 Getting your data + feature engineering

In [5]:
data = pd.read_csv("bank_transactions.csv")

In [6]:
data

,TransactionID,CustomerID,CustomerDOB,CustGender,CustLocation,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmount (INR)
0,T1,C5841053,10/1/94,F,JAMSHEDPUR,17819.05,2/8/16,143207,25.0
1,T2,C2142763,4/4/57,M,JHAJJAR,2270.69,2/8/16,141858,27999.0
2,T3,C4417068,26/11/96,F,MUMBAI,17874.44,2/8/16,142712,459.0
3,T4,C5342380,14/9/73,F,MUMBAI,866503.21,2/8/16,142714,2060.0
4,T5,C9031234,24/3/88,F,NAVI MUMBAI,6714.43,2/8/16,181156,1762.5
...,...,...,...,...,...,...,...,...,...
1048562,T1048563,C8020229,8/4/90,M,NEW DELHI,7635.19,18/9/16,184824,799.0
1048563,T1048564,C6459278,20/2/92,M,NASHIK,27311.42,18/9/16,183734,460.0
1048564,T1048565,C6412354,18/5/89,M,HYDERABAD,221757.06,18/9/16,183313,770.0
1048565,T1048566,C6420483,30/8/78,M,VISAKHAPATNAM,10117.87,18/9/16,184706,1000.0


We created the seven features. They are the following: <br><br>
    <b>1. no_of_transactions</b> = Number of transactions executed by each customer<br>
    <b>2. no_of_big_transactions</b> = Number of transactions with a balance bigger than 100 dollars<br>
    <b>3. avg_transaction_amount</b> = The average amount of each transfer by each customer<br>
    <b>4. avg_balance</b> = Average balance of each customer<br>
    <b>5. utilisation</b> = Average difference between the balance and the transaction amount for each customer<br>
    <b>6. most_common_gender</b>  = Most common gender of each customer<br>
    <b>7. most_common_location</b>  = Most common location of each customer<br>

In [77]:
data2 = data.groupby(['CustomerID'])['CustomerID'].count().reset_index(name='no_of_transactions')
data3 = data.groupby(['CustomerID'])["TransactionAmount (INR)"].apply(lambda x: (x>100.0).sum()).reset_index(name='no_of_big_transactions')
data_final = pd.merge(data2,data3, on="CustomerID")

data4= data.groupby(['CustomerID'])["TransactionAmount (INR)"].mean().reset_index(name='avg_transaction_amount')
data_final = pd.merge(data_final,data4, on="CustomerID")

data5= data.groupby(['CustomerID'])["CustAccountBalance"].mean().reset_index(name='avg_balance')
data_final = pd.merge(data_final,data5, on="CustomerID")

data6 = data[["CustomerID", "CustAccountBalance", "TransactionAmount (INR)"]]
data6['diff'] = data6["CustAccountBalance"] - data6["TransactionAmount (INR)"]
data6 = data6.groupby(["CustomerID"])["diff"].mean().reset_index(name='utilisation')
data_final = pd.merge(data_final,data6, on="CustomerID")

data7= data.groupby(['CustomerID'])['CustGender'].agg(pd.Series.mode).reset_index(name='most_common_gender')
data_final = pd.merge(data_final,data7, on="CustomerID")

data8= data.groupby(['CustomerID'])['CustLocation'].agg(pd.Series.mode).reset_index(name='most_common_location')
data_final = pd.merge(data_final,data8, on="CustomerID")

C:\Users\borit\AppData\Local\Temp\ipykernel_3652\3690358859.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data6['diff'] = data6["CustAccountBalance"] - data6["TransactionAmount (INR)"]


In [78]:
data_final

,CustomerID,no_of_transactions,no_of_big_transactions,avg_transaction_amount,avg_balance,utilisation,most_common_gender,most_common_location
0,C1010011,2,2,2553.0,76340.635,73787.635,"[F, M]","[NEW DELHI, NOIDA]"
1,C1010012,1,1,1499.0,24204.490,22705.490,M,MUMBAI
2,C1010014,2,2,727.5,100112.950,99385.450,"[F, M]",MUMBAI
3,C1010018,1,0,30.0,496.180,466.180,F,CHAMPARAN
4,C1010024,1,1,5000.0,87058.650,82058.650,M,KOLKATA
...,...,...,...,...,...,...,...,...
884260,C9099836,1,1,691.0,133067.230,132376.230,M,BHIWANDI
884261,C9099877,1,1,222.0,96063.460,95841.460,M,BANGALORE
884262,C9099919,1,1,126.0,5559.750,5433.750,M,GUNTUR
884263,C9099941,1,0,50.0,35295.920,35245.920,M,CHENNAI


We created 20 custom features. They are the following: <br>
<b>1. avg_transaction_time</b> = The average time of the transactions for each customer<br>
2.<br>
3.<br>
4.<br>
5.<br>
6.<br>
7.<br>
8.<br>
9.<br>
10.<br>
11.<br>
12.<br>
13.<br>
14.<br>
15.<br>
16.<br>
17.<br>
18.<br>
19.<br>
20.

In [79]:
feature= data.groupby(['CustomerID'])["TransactionTime"].mean().reset_index(name='avg_transaction_time')
data_final = pd.merge(data_final,feature, on="CustomerID")

In [80]:
data_final

,CustomerID,no_of_transactions,no_of_big_transactions,avg_transaction_amount,avg_balance,utilisation,most_common_gender,most_common_location,avg_transaction_time
0,C1010011,2,2,2553.0,76340.635,73787.635,"[F, M]","[NEW DELHI, NOIDA]",67521.0
1,C1010012,1,1,1499.0,24204.490,22705.490,M,MUMBAI,204409.0
2,C1010014,2,2,727.5,100112.950,99385.450,"[F, M]",MUMBAI,187378.0
3,C1010018,1,0,30.0,496.180,466.180,F,CHAMPARAN,170254.0
4,C1010024,1,1,5000.0,87058.650,82058.650,M,KOLKATA,141103.0
...,...,...,...,...,...,...,...,...,...
884260,C9099836,1,1,691.0,133067.230,132376.230,M,BHIWANDI,5122.0
884261,C9099877,1,1,222.0,96063.460,95841.460,M,BANGALORE,120255.0
884262,C9099919,1,1,126.0,5559.750,5433.750,M,GUNTUR,122533.0
884263,C9099941,1,0,50.0,35295.920,35245.920,M,CHENNAI,213722.0
